In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datatable as dt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc
import warnings
warnings.filterwarnings("ignore")
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv
/kaggle/input/playground-series-s3e2/sample_submission.csv
/kaggle/input/playground-series-s3e2/train.csv
/kaggle/input/playground-series-s3e2/test.csv


In [2]:
train = pd.read_csv('../input/playground-series-s3e2/train.csv', index_col=0);
train.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0


In [3]:
test = pd.read_csv('../input/playground-series-s3e2/test.csv', index_col=0);
test.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
id,,,,,,,,,,
15304,Female,57.0,0,0,Yes,Private,Rural,82.54,33.4,Unknown
15305,Male,70.0,1,0,Yes,Private,Urban,72.06,28.5,Unknown
15306,Female,5.0,0,0,No,children,Urban,103.72,19.5,Unknown
15307,Female,56.0,0,0,Yes,Govt_job,Urban,69.24,41.4,smokes
15308,Male,32.0,0,0,Yes,Private,Rural,111.15,30.1,smokes


This dataset is external to this competition, and is what this competition's synthetic data is based upon. The features in the dataset below are exactly the same as in this competition's.

In [4]:
extend_df = pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv', index_col=0);
extend_df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
id,,,,,,,,,,,
9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [5]:
r1,c1 = train.shape
print('The training data has {} rows and {} columns'.format(r1,c1))
r2,c2 = test.shape
print('The test data has {} rows and {} columns'.format(r2,c2))
r3,c3 = extend_df.shape
print('The extend data has {} rows and {} columns'.format(r3,c3))

The training data has 15304 rows and 11 columns
The test data has 10204 rows and 10 columns
The extend data has 5110 rows and 11 columns


In [6]:
# To see the quantity of null vaues in all the columns.
# c1 stands for the number of columns in the training data.

print('MISSING VALUES IN TRAINING DATASET:')
print(train.isna().sum().nlargest(c1))
print('')
print('MISSING VALUES IN TEST DATASET:')
print(test.isna().sum().nlargest(c2))
print('')
print('MISSING VALUES IN EXTEND DATASET:')
print(test.isna().sum().nlargest(c3))

MISSING VALUES IN TRAINING DATASET:
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

MISSING VALUES IN TEST DATASET:
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
dtype: int64

MISSING VALUES IN EXTEND DATASET:
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
dtype: int64


In [7]:
def feature_risk_factors(df):
    df['blood_glucose'] = df['avg_glucose_level'].apply(lambda x: 'normal' if x <= 99 else 'prediabetic' if x <= 125 else 'diabetic')
    df['bmi_cat'] = df['bmi'].apply(lambda x: 'underweight' if x < 18.5 else 'normal' if x <= 24.99 else 'overweight' if x <=29.99 else 'obese category 1' if x <= 34.99 else 'obese category 2' if x <= 39.99 else 'obese category 3')
    df['heart_disease_risk'] = df['age'].apply(lambda x: 'low risk' if x < 45 else 'high risk' if x < 65 else 'unknown')
    df['risk_factors'] = df[['avg_glucose_level', 'age', 'bmi', 'hypertension', 'heart_disease', 'smoking_status']].apply(
        lambda x: 0 + \
            (1 if x['avg_glucose_level'] > 99 else 0) + \
            (1 if x['age'] > 45 else 0) + \
            (1 if x['bmi'] > 24.99 else 0) + \
            (1 if x['hypertension'] == 1 else 0) + \
            (1 if x['smoking_status'] in ['formerly smoked', 'smokes'] else 0),
        axis=1
    )
    return df

In [8]:
train = feature_risk_factors(train)
test = feature_risk_factors(test)
extend_df = feature_risk_factors(extend_df)

In [9]:
test.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,blood_glucose,bmi_cat,heart_disease_risk,risk_factors
id,,,,,,,,,,,,,,
15304,Female,57.0,0,0,Yes,Private,Rural,82.54,33.4,Unknown,normal,obese category 1,high risk,2
15305,Male,70.0,1,0,Yes,Private,Urban,72.06,28.5,Unknown,normal,overweight,unknown,3
15306,Female,5.0,0,0,No,children,Urban,103.72,19.5,Unknown,prediabetic,normal,low risk,1
15307,Female,56.0,0,0,Yes,Govt_job,Urban,69.24,41.4,smokes,normal,obese category 3,high risk,3
15308,Male,32.0,0,0,Yes,Private,Rural,111.15,30.1,smokes,prediabetic,obese category 1,low risk,3


In [10]:
train_df = pd.concat([train, extend_df]); 
train_df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,blood_glucose,bmi_cat,heart_disease_risk,risk_factors
id,,,,,,,,,,,,,,,
0,Male,28.0,0,0,Yes,Private,Urban,79.53,31.1,never smoked,0,normal,obese category 1,low risk,1
1,Male,33.0,0,0,Yes,Private,Rural,78.44,23.9,formerly smoked,0,normal,normal,low risk,1
2,Female,42.0,0,0,Yes,Private,Rural,103.00,40.3,Unknown,0,prediabetic,obese category 3,low risk,2
3,Male,56.0,0,0,Yes,Private,Urban,64.87,28.8,never smoked,0,normal,overweight,high risk,2
4,Female,24.0,0,0,No,Private,Rural,73.36,28.8,never smoked,0,normal,overweight,low risk,1


In [11]:
test.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,blood_glucose,bmi_cat,heart_disease_risk,risk_factors
id,,,,,,,,,,,,,,
15304,Female,57.0,0,0,Yes,Private,Rural,82.54,33.4,Unknown,normal,obese category 1,high risk,2
15305,Male,70.0,1,0,Yes,Private,Urban,72.06,28.5,Unknown,normal,overweight,unknown,3
15306,Female,5.0,0,0,No,children,Urban,103.72,19.5,Unknown,prediabetic,normal,low risk,1
15307,Female,56.0,0,0,Yes,Govt_job,Urban,69.24,41.4,smokes,normal,obese category 3,high risk,3
15308,Male,32.0,0,0,Yes,Private,Rural,111.15,30.1,smokes,prediabetic,obese category 1,low risk,3


In [12]:
print('unique value of categorical valiables')

# categorical valiables
col = ['gender','hypertension','heart_disease','ever_married','work_type','Residence_type',
       'smoking_status','risk_factors','blood_glucose','bmi_cat','heart_disease_risk']

print('\n------- train -------\n')

for i in col:
    print(f'{i} : {train_df[i].unique()}')

print('\n------- test  -------\n')

for i in col:
    print(f'{i} : {test[i].unique()}')

unique value of categorical valiables

------- train -------

gender : ['Male' 'Female' 'Other']
hypertension : [0 1]
heart_disease : [0 1]
ever_married : ['Yes' 'No']
work_type : ['Private' 'Self-employed' 'Govt_job' 'children' 'Never_worked']
Residence_type : ['Urban' 'Rural']
smoking_status : ['never smoked' 'formerly smoked' 'Unknown' 'smokes']
risk_factors : [1 2 0 3 4 5]
blood_glucose : ['normal' 'prediabetic' 'diabetic']
bmi_cat : ['obese category 1' 'normal' 'obese category 3' 'overweight' 'underweight'
 'obese category 2']
heart_disease_risk : ['low risk' 'high risk' 'unknown']

------- test  -------

gender : ['Female' 'Male' 'Other']
hypertension : [0 1]
heart_disease : [0 1]
ever_married : ['Yes' 'No']
work_type : ['Private' 'children' 'Govt_job' 'Self-employed' 'Never_worked']
Residence_type : ['Rural' 'Urban']
smoking_status : ['Unknown' 'smokes' 'never smoked' 'formerly smoked']
risk_factors : [2 3 1 0 4 5]
blood_glucose : ['normal' 'prediabetic' 'diabetic']
bmi_cat : ['

In [13]:
from sklearn import preprocessing
def encode_features(df_train, df_test):
    features = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'blood_glucose','bmi_cat','heart_disease_risk']
    df_combined = pd.concat([df_train[features], df_test[features]])
    
    for feature in features:
        le = preprocessing.LabelEncoder()
        le = le.fit(df_combined[feature])
        df_train[feature] = le.transform(df_train[feature])
        df_test[feature] = le.transform(df_test[feature])
    return df_train, df_test
    
train_df, test = encode_features(train_df, test)


In [14]:
train_df.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke,blood_glucose,bmi_cat,heart_disease_risk,risk_factors
id,,,,,,,,,,,,,,,
0,1,28.0,0,0,1,2,1,79.53,31.1,2,0,1,1,1,1
1,1,33.0,0,0,1,2,0,78.44,23.9,1,0,1,0,1,1
2,0,42.0,0,0,1,2,0,103.00,40.3,0,0,2,3,1,2
3,1,56.0,0,0,1,2,1,64.87,28.8,2,0,1,4,0,2
4,0,24.0,0,0,0,2,0,73.36,28.8,2,0,1,4,1,1


In [15]:
test.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,blood_glucose,bmi_cat,heart_disease_risk,risk_factors
id,,,,,,,,,,,,,,
15304,0,57.0,0,0,1,2,0,82.54,33.4,0,1,1,0,2
15305,1,70.0,1,0,1,2,1,72.06,28.5,0,1,4,2,3
15306,0,5.0,0,0,0,4,1,103.72,19.5,0,2,0,1,1
15307,0,56.0,0,0,1,0,1,69.24,41.4,3,1,3,0,3
15308,1,32.0,0,0,1,2,0,111.15,30.1,3,2,1,1,3


In [16]:
X = train_df.drop('stroke', axis=1).copy()
y = train_df['stroke'].copy()
X_test = test.copy()

del train_df
del test

In [17]:
params = {
    'max_depth':6,
    'n_estimators': 8500,
    'subsample': 0.85,
    'colsample_bytree': 0.20,
    'colsample_bylevel': 0.60,
    'min_child_weight':50,
    'reg_lambda': 75,
    'reg_alpha': 0.15,
    'gamma': 0.75,
    'booster': 'gbtree',
    'eval_metric': 'auc',
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'use_label_encoder': False
    }

In [18]:
%%time
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

preds = []
scores = []

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    params['learning_rate']=0.007
    model1 = XGBClassifier(**params)
    model1.fit(X_train,y_train,
              eval_set=[(X_train, y_train),(X_valid,y_valid)],
              early_stopping_rounds=200,
              verbose=False)
    
    params['learning_rate']=0.03
    model2 = XGBClassifier(**params)
    model2.fit(X_train,y_train,
              eval_set=[(X_train, y_train),(X_valid,y_valid)],
              early_stopping_rounds=200,
              verbose=False,
              xgb_model=model1)
    
    params['learning_rate']=0.1
    model3 = XGBClassifier(**params)
    model3.fit(X_train,y_train,
              eval_set=[(X_train, y_train),(X_valid,y_valid)],
              early_stopping_rounds=200,
              verbose=False,
              xgb_model=model2)
    
    pred_valid = model3.predict_proba(X_valid)[:,1]
    fpr, tpr, _ = roc_curve(y_valid, pred_valid)
    score = auc(fpr, tpr)
    scores.append(score)
    
    print(f"Fold: {fold + 1} Score: {score}")
    print('._'*30)
    
    test_preds = model3.predict_proba(X_test)[:,1]
    preds.append(test_preds)
    
print(f"Overall Validation Score: {np.mean(scores)}")

Fold: 1 Score: 0.874888021499872
._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._
Fold: 2 Score: 0.882058281173651
._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._
Fold: 3 Score: 0.8664161935453848
._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._
Fold: 4 Score: 0.8884672573846986
._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._
Fold: 5 Score: 0.8724551098543034
._._._._._._._._._._._._._._._._._._._._._._._._._._._._._._
Overall Validation Score: 0.8768569726915821
CPU times: user 1min 13s, sys: 1.21 s, total: 1min 14s
Wall time: 1min 14s


In [19]:
predictions = np.mean(np.column_stack(preds),axis=1)
ss = pd.read_csv('../input/playground-series-s3e2/sample_submission.csv',);
ss['stroke'] = predictions
ss.to_csv('sub-final.csv', index=False)
ss.head()

,id,stroke
0,15304,0.040937
1,15305,0.218730
2,15306,0.002002
3,15307,0.055514
4,15308,0.006319
